# EKSTRAK TEKS PDF

In [1]:
!curl https://www.gov.br/receitafederal/pt-br/centrais-de-conteudo/publicacoes/perguntas-e-respostas/dirf/perguntas-e-respostas-dirf-2022/@@download/file --output itr22.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  608k  100  608k    0     0   239k      0  0:00:02  0:00:02 --:--:--  239k


In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


ekstrak teks dalam pdf

In [3]:
from PyPDF2 import PdfReader
from pathlib import Path
import pandas as pd
import re
from re import search
# create pandas dataframe to store the QnAs extracted from pdf files
df = pd.DataFrame()

filename = 'ITR2022.pdf'  # pdf file to be parsed
skipto = 12
qa = []

# read file
reader = PdfReader(filename)
# get total number of pages
num_pages = len(reader.pages)
print('File %s has %d pages' % (filename, num_pages))

question = ''
answer = ''
skipline = False
stillquestion = False

qnum = "|".join("^%03d —" % i for i in range(227))

for idx in range(skipto, num_pages):
    page = reader.pages[idx]
    pagetext = page.extract_text()

    lines = pagetext.split('\n')
    lineidx = 0
    while lineidx < len(lines):
        line = lines[lineidx]
        if search(qnum, line):
            if question:
                qa.append({'question': question, 'answer': answer})
            question = line.split('—')[1]
            answer = ''
            if '?' not in line:
                stillquestion = True
        elif 'Retorno ao sumário' in line:
            lineidx += 1
            if lineidx >= len(lines):
                break
            line = lines[lineidx]
            if search(qnum, line):
                continue
        elif '?' in line:
            question = question + ' ' + line
            stillquestion = False
        else:
            if stillquestion:
                question = question + ' ' + line
            else:
                answer = answer + ' ' + line
        lineidx += 1
qa.append({'question': question, 'answer': answer})

print('we have a total of %d questions and answers for the ITR 2022 pdf file' % len(qa))
df_temp = pd.DataFrame.from_dict(qa)

df = pd.concat([df, df_temp])

# saving QnA data
df.to_csv('LegalQA_dataset.csv', index=False, sep='\t')

File ITR2022.pdf has 81 pages
we have a total of 226 questions and answers for the ITR 2022 pdf file


# BUAT FORMAT QNA

template prompt untuk input question answering

```
"Given the question delimited by triple backticks ```{question}```,
what is the answer? Answer: {answer}"
```

In [4]:
import pandas as pd
import json

# load portuguese legal QnA datasetL LegalQA
dataset = pd.read_csv("LegalQA_dataset.csv", sep = '\t')
dataset

,question,answer
0,O que é o ITR?,ITR é o Imposto sobre a Propriedade Territori...
1,Qual é o fato gerador do ITR?,"O fato gerador do ITR é a propriedade, o domí..."
2,Qual é o período de apuração do ITR?,O período de apuração do ITR é anual. (Lei ...
3,O ITR incide sobre imóvel rural desapropriado...,Sim. O ITR incide sobre a propriedade rural d...
4,O ITR incide sobre imóvel rural desapropriado...,Sim. Quando a desapropriação for promovida po...
...,...,...
221,Quais os acréscimos legais incidentes sobre o...,A falta ou insuficiência de pagamento do impo...
222,Quais os municípios que formam a Amazônia Oci...,Os municípi os que formam a Amazônia Ocidenta...
223,Quais os municípios que formam o Pantanal mat...,Pertencem ao Pantanal mato -grossense e sul -...
224,Quais os municípios que formam a Amazônia Ori...,Os municípios que formam a Amazônia Oriental ...


In [5]:
def buildprompt(data):
    prompt = {}
    prompt['text'] = "Given the question delimited by triple backticks ```{" + data['question'] + "}```, what is the answer? Answer: {" + data['answer'] + "}"
    return prompt

In [6]:
dataset['prompt'] = dataset.apply(buildprompt, axis=1)

In [7]:
print(dataset['prompt'])

0      {'text': 'Given the question delimited by trip...
1      {'text': 'Given the question delimited by trip...
2      {'text': 'Given the question delimited by trip...
3      {'text': 'Given the question delimited by trip...
4      {'text': 'Given the question delimited by trip...
                             ...                        
221    {'text': 'Given the question delimited by trip...
222    {'text': 'Given the question delimited by trip...
223    {'text': 'Given the question delimited by trip...
224    {'text': 'Given the question delimited by trip...
225    {'text': 'Given the question delimited by trip...
Name: prompt, Length: 226, dtype: object


In [8]:
# save prompts to a json file
result = dataset['prompt'].to_list()
with open('prompts.json', 'w') as outfile:
    json.dump(result, outfile, ensure_ascii=False)

# BUILD DAN TRAIN MODEL BLOOM

In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
import pandas as pd
import torch
import json
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from accelerate import Accelerator

In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [12]:
# Loading bloomz model and tokenizer
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloomz-560m").to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [13]:
# Loading dataset prompts.json built using de portuguese legalQA dataset
dataset = load_dataset("json", data_files="prompts.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 226
    })
})

data preparation

In [30]:
# prepare the data for training
def prepare_train_data(data):
    # prompt + completion
    #text_input = data['prompt'] + ' ' + data['completion']
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(text_input, return_tensors='pt', padding=True)
    # generative models: labels are the same as the input
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

In [31]:
train_dataset = dataset['train'].map(prepare_train_data, batched=True, remove_columns=["text"])

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

In [32]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 226
})

In [18]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [41]:
# setting arguments to be used during training
training_arguments = TrainingArguments(
    'LegalQA-bloom-560m',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    optim="adafactor",
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [42]:
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset
)

In [ ]:
# training
trainer.train()

In [ ]:
# simpan model
trainer.save_model()

# inference

In [ ]:
import torch
from transformers import pipeline
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer

In [ ]:
# Loading the fine-tuned model: LegalQA-bloom-560m
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("LegalQA-bloom-560m",low_cpu_mem_usage=True).to("cpu")
prompt = 'Given the question delimited by triple backticks ```{ O que é o imposto territorial rural?  }```, what is the answer? Answer:'
generator = pipeline('text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt, max_length=128)
print(result)

In [ ]:
# Loading the original model: bloomz-560m
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloomz-560m",low_cpu_mem_usage=True).to("cpu")
prompt = 'Given the question delimited by triple backticks ```{ O que é o imposto territorial rural?  }```, what is the answer? Answer:'
generator = pipeline('text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt, max_length=128)
print(result)